In [1]:
import sys
sys.path.append('../../')

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

import zipfile
import os
from pathlib import Path
import urllib.request


In [2]:
spark = SparkSession.builder \
    .appName("ner")\
    .master("local[1]")\
    .config("spark.driver.memory","8G")\
    .config("spark.driver.maxResultS1ize", "2G") \
    .config("spark.jars.packages", "JohnSnowLabs:spark-nlp:2.0.0")\
    .config("spark.kryoserializer.buffer.max", "500m")\
    .getOrCreate()

1. Download CoNLL2003 dataset
2. Save 3 files eng.train, eng.testa, eng.testa, into working dir ./

In [3]:
# Example how to download CoNLL 2003 Dataset
def download_conll2003_file(file):    
    if not Path(file).is_file():
        url = "https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/" + file
        urllib.request.urlretrieve(url, file)
        
download_conll2003_file("eng.train")
download_conll2003_file("eng.testa")
download_conll2003_file("eng.testb")

3 Download Glove word embeddings

In [4]:
file = "glove.6B.zip"
if not Path("glove.6B.zip").is_file():
    url = "http://nlp.stanford.edu/data/glove.6B.zip"
    print("Start downoading Glove Word Embeddings. It will take some time, please wait...")
    urllib.request.urlretrieve(url, "glove.6B.zip")
    print("Downloading finished")
    
if not Path("glove.6B.100d.txt").is_file():
    zip_ref = zipfile.ZipFile(file, 'r')
    zip_ref.extractall("./")
    zip_ref.close()


In [5]:
import time


def get_pipeline():
    glove = WordEmbeddingsLookup()\
      .setInputCols(["sentence", "token"])\
      .setOutputCol("glove")\
      .setEmbeddingsSource("glove.6B.100d.txt", 100, 2)

    nerTagger = NerCrfApproach()\
      .setInputCols(["sentence", "token", "pos", "glove"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMinEpochs(1)\
      .setMaxEpochs(10)\
      .setLossEps(1e-3)\
      .setExternalFeatures("../../../src/test/resources/ner-corpus/dict.txt", ",")\
      .setL2(1)\
      .setC0(1250000)\
      .setRandomSeed(100)\
      .setVerbose(2)
      
    pipeline = Pipeline(
        stages = [
        glove,
        nerTagger
      ])
    
    return pipeline

def readDataset(file):
    from sparknlp.dataset import CoNLL
    conll = CoNLL()
    return conll.readDataset('eng.train')
    
def train_model(file):
    global spark
    
    print("Dataset Reading")
    
    start = time.time()
    dataset = readDataset(file)
    print("Done, {}\n".format(time.time() - start))

    print("Start fitting")
    pipeline = get_pipeline()

    return pipeline.fit(dataset)
 


In [16]:
from pyspark.sql.functions import col, udf, explode
from pyspark.sql.types import *


annotation_schema = StructType([
    StructField("annotatorType", StringType()),
    StructField("begin", IntegerType(), False),
    StructField("end", IntegerType(), False),
    StructField("result", StringType()),
    StructField("metadata", MapType(StringType(), StringType())),
    StructField("calculations", MapType(StringType(), ArrayType(FloatType()))),
])


def get_dataset_for_analysis(file, model):
    global spark
    
    print("Dataset Reading")
    
    start = time.time()
    dataset = readDataset(file)
    print("Done, {}\n".format(time.time() - start))
    
    predicted = model.transform(dataset)
    
    global annotation_schema
    
    zip_annotations = udf(
      lambda x, y: list(zip(x, y)),
      ArrayType(StructType([
          StructField("predicted", annotation_schema),
          StructField("label", annotation_schema)
      ]))
    )
    
    return predicted\
        .withColumn("result", zip_annotations("ner", "label"))\
        .select(explode("result").alias("result"))\
        .select(
            col("result.predicted").alias("predicted"), 
            col("result.label").alias("label")
        )
        
def printStat(label, correct, predicted, predictedCorrect):
    prec = predictedCorrect / predicted if predicted > 0 else 0
    rec = predictedCorrect / correct if correct > 0 else 0
    f1 = (2*prec*rec)/(prec + rec) if prec + rec > 0 else 0
    
    print("{}\t{}\t{}\t{}".format(label, prec, rec, f1))
        

def test_dataset(file, model, ignore_tokenize_misses=True):
    global spark
    
    started = time.time()

    df = readDataset(file)
    transformed = model.transform(df).select("label", "ner")

    labels = []
    predictedLabels = []

    for line in transformed.collect():
        label = line[0]
        ner = line[1]
    
        ner = {(a["begin"], a["end"]):a["result"] for a in ner}

        for a in label:
            key = (a["begin"], a["end"])

            label = a["result"].strip()
            predictedLabel = ner.get(key, "O").strip()
            
            if key not in ner and ignore_tokenize_misses:
                continue
                
            labels.append(label)
            predictedLabels.append(predictedLabel)
        

    correct = {}
    predicted = {}
    predictedCorrect = {}


    print(len(labels))

    for (lPredicted, lCorrect) in zip(predictedLabels, labels):
        correct[lCorrect] = correct.get(lCorrect, 0) + 1
        predicted[lPredicted] = predicted.get(lPredicted, 0) + 1

        if lCorrect == lPredicted:
            predictedCorrect[lPredicted] = predictedCorrect.get(lPredicted, 0) + 1

    correct = { key: correct[key] for key in correct.keys() if key != 'O'}
    predicted = { key: predicted[key] for key in predicted.keys() if key != 'O'}
    predictedCorrect = { key: predictedCorrect[key] for key in predictedCorrect.keys() if key != 'O'}

    tags = set(list(correct.keys()) + list(predicted.keys()))

    print("label\tprec\trec\tf1")
    totalCorrect = sum(correct.values())
    totalPredicted = sum(predicted.values())
    totalPredictedCorrect = sum(predictedCorrect.values())

    printStat("Total", totalCorrect, totalPredicted, totalPredictedCorrect)

    for label in tags:
        printStat(label, correct.get(label, 0), predicted.get(label, 0), predictedCorrect.get(label, 0))


SyntaxError: invalid syntax (<ipython-input-16-e638fc6d6114>, line 5)

In [7]:
import os.path

folder = '.'
train_file = os.path.join(folder, "eng.train")
test_file_a = os.path.join(folder, "eng.testa")
test_file_b = os.path.join(folder, "eng.testb")

In [8]:
model = train_model(train_file)

Dataset Reading
Done, 4.746060132980347

Start fitting


In [9]:
print("\nQuality on training data")
test_dataset(train_file, model)

print("\n\nQuality on validation data")
test_dataset(test_file_a, model)

print("\n\nQuality on test data")
test_dataset(test_file_b, model)


Quality on training data
203621
label	prec	rec	f1
Total	0.9698192717452592	0.9599624004934936	0.9648656628284618
I-LOC	0.9827103372701814	0.9740526188752112	0.9783623249893934
B-ORG	1.0	1.0	1.0
I-ORG	0.989902315881901	0.9018098190180982	0.9438049393051486
B-LOC	1.0	0.7272727272727273	0.8421052631578948
I-PER	0.9843444227005871	0.9944284687275341	0.9893607510058112
I-MISC	0.880746561886051	0.9839771729587358	0.9295044578063446
B-MISC	1.0	0.24324324324324326	0.391304347826087


Quality on validation data
203621
label	prec	rec	f1
Total	0.9698192717452592	0.9599624004934936	0.9648656628284618
I-LOC	0.9827103372701814	0.9740526188752112	0.9783623249893934
B-ORG	1.0	1.0	1.0
I-ORG	0.989902315881901	0.9018098190180982	0.9438049393051486
B-LOC	1.0	0.7272727272727273	0.8421052631578948
I-PER	0.9843444227005871	0.9944284687275341	0.9893607510058112
I-MISC	0.880746561886051	0.9839771729587358	0.9295044578063446
B-MISC	1.0	0.24324324324324326	0.391304347826087


Quality on test data
203621
label	p

In [14]:
df = get_dataset_for_analysis(test_file_a, model)
df.show()

Dataset Reading
Done, 1.554076910018921



NameError: name 'annotation_schema' is not defined

In [ ]:
get_pipeline().write().overwrite().save("./crf_pipeline")
model.write().overwrite().save("./crf_model")

In [ ]:
from pyspark.ml import PipelineModel, Pipeline

Pipeline.read().load("./crf_pipeline")
sameModel = PipelineModel.read().load("./crf_model")

In [ ]:
print("\nQuality on training data")
test_dataset(train_file, sameModel)

print("\n\nQuality on validation data")
test_dataset(test_file_a, sameModel)

print("\n\nQuality on test data")
test_dataset(test_file_b, sameModel)